In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
from scipy import stats
import os

os.chdir("../Projet-CMI-Calcul-Num")

# Election présidentielle 2022 et variables socio-démographiques dans les Hauts-de-Seine

données :

- Données des élections présidentielles françaises de 2022.
- Variables socio demographiques par commune française
- Variables socio démographiques par département français

In [9]:
# Chargement des données

pres22 = pd.read_csv("../Projet-CMI-Calcul-Num/data/pres2022comm.csv", dtype={'codecommune': str, 'dep': str})
socio_demo = pd.read_csv("../Projet-CMI-Calcul-Num/data/var_socio_demo.csv", dtype={'codecommune': str, 'dep': str})
socio_dep = pd.read_csv("../Projet-CMI-Calcul-Num/data/var_socio_dep.csv", dtype={'codecommune': str, 'dep': str})


print(socio_demo.columns)
print(socio_dep.columns)


Index(['codecommune', 'agri2022', 'indp2022', 'cadr2022', 'pint2022',
       'empl2022', 'ouvr2022', 'chom2022', 'aind2022', 'aica2022',
       ...
       'propf60p2022', 'prop0142022', 'prop15392022', 'prop40592022',
       'prop60p2022', 'perpropf2022', 'perage2022', 'perprop0142022',
       'perprop60p2022', 'pimmigre2022'],
      dtype='object', length=105)
Index(['dep', 'revratio2022', 'revratioadu2022', 'revratiofoy2022',
       'revtot2022', 'pop2022_x', 'nadult2022', 'nfoyer2022', 'revmoyfoy2022',
       'revmoyadu2022', 'revmoy2022', 'pibhab2022', 'pib2022', 'pop2022_y',
       'nodiph2022', 'nodipf2022', 'nodip2022', 'bach2022', 'bacf2022',
       'bac2022', 'suph2022', 'supf2022', 'sup2022', 'pbac2022', 'psup2022',
       'popf2022', 'poph2022', 'popf0142022', 'popf15392022', 'popf40592022',
       'popf60p2022', 'poph0142022', 'poph15392022', 'poph40592022',
       'poph60p2022', 'agef2022', 'ageh2022', 'age2022', 'pop2022',
       'propf2022', 'propf0142022', 'propf1539202

### Partie 1. Statistiques descriptives sur le département

In [1]:
# Statistiques descriptives